In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [6]:
df = pd.read_csv('data/tictactoe_dataset.csv')
df['board_list'] = df['board_state'].apply(list)

# === 3. Encodage des caractères du plateau ('X', 'O', ' ') ===
le_board = LabelEncoder()
all_cells = sum(df['board_list'].tolist(), [])  # aplatit la liste
le_board.fit(all_cells)

X_encoded = [le_board.transform(row) for row in df['board_list']]
# === 4. Encodage des coups 'i,j' ===
le_move = LabelEncoder()
y_encoded = le_move.fit_transform(df['best_move'])

In [ ]:


# === 5. Split train/test ===
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

# === 6. Entraînement du modèle ===
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train, y_train)

# === 7. Évaluation ===
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Précision du modèle : {acc:.2f}")

def predict_best_move(board_list, model, le_board, le_move):
    """
    board_list : liste de 9 caractères ['X', 'O', ' ', ...]
    model      : le modèle entraîné (ex: clf)
    le_board   : LabelEncoder entraîné sur les cases du plateau
    le_move    : LabelEncoder entraîné sur les coups à jouer ('0,0', etc.)

    return : meilleur coup sous forme de string 'i,j'
    """
    if len(board_list) != 9:
        raise ValueError("Le plateau doit contenir exactement 9 cases.")
    
    encoded = le_board.transform(board_list).reshape(1, -1)
    move_encoded = model.predict(encoded)
    move = le_move.inverse_transform(move_encoded)[0]
    return move

def print_board(b):
    for i in range(3):
        print(" | ".join(b[i*3:(i+1)*3]))
        if i < 2:
            print("-" * 10)


board = ['O', ' ', ' ',
         ' ', 'X', ' ',
         ' ', ' ', ' ']

print_board(board)
print("X centre, O coin :", predict_best_move(board, clf, le_board, le_move))




Précision du modèle : 0.64
O |   |  
-----
  | X |  
-----
  |   |  
X centre, O coin : 0,1
